In [12]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from tqdm import tqdm
import numpy as np

In [13]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [14]:
df = pd.read_csv("data/spotify_millsongdata_w_genres.csv")

In [15]:
song_embeddings = model.encode(df['song'].values, show_progress_bar=True)
lyrics_embeddings = model.encode(df['cleaned_text'].values, show_progress_bar=True)
genres_embeddings = model.encode(df['genres'].values, show_progress_bar=True)

Batches:   0%|          | 0/782 [00:00<?, ?it/s]

Batches:   0%|          | 0/782 [00:00<?, ?it/s]

Batches:   0%|          | 0/782 [00:00<?, ?it/s]

In [16]:
embeddings = []
for i in range(len(song_embeddings)):
    embeddings.append((song_embeddings[i] + lyrics_embeddings[i] + genres_embeddings[i]) / 3)

In [17]:
cos_sim = util.cos_sim(embeddings, embeddings)

In [18]:
similarity_df = pd.DataFrame(cos_sim)

In [19]:
similarity_df.head()

,0,1,2,3,4,5,6,7,8,9,...,24990,24991,24992,24993,24994,24995,24996,24997,24998,24999
0,1.000000,0.388779,0.521183,0.556013,0.539055,0.462960,0.424032,0.449598,0.553453,0.455835,...,0.539191,0.517333,0.490352,0.489927,0.451889,0.443245,0.389885,0.519190,0.512235,0.411215
1,0.388779,1.000000,0.504900,0.501146,0.472421,0.355256,0.417171,0.495605,0.437213,0.425212,...,0.497644,0.382998,0.455469,0.348523,0.474053,0.476317,0.317961,0.465851,0.476618,0.379288
2,0.521183,0.504900,1.000000,0.574440,0.572905,0.509668,0.462410,0.480853,0.536980,0.517986,...,0.607825,0.514046,0.593987,0.523652,0.490726,0.453753,0.413252,0.536125,0.615163,0.473930
3,0.556013,0.501146,0.574440,1.000000,0.882615,0.483101,0.488258,0.493010,0.566492,0.538338,...,0.568736,0.509073,0.567920,0.475656,0.513535,0.464334,0.423422,0.562718,0.496793,0.537681
4,0.539055,0.472421,0.572905,0.882615,1.000000,0.453808,0.470191,0.508774,0.546013,0.488342,...,0.584105,0.474550,0.534472,0.462861,0.501672,0.446204,0.416900,0.525003,0.535542,0.516275


In [20]:
similarity_df.to_csv('data/similarity_matrix.csv', index=False)

In [21]:
song_index = 50

In [22]:
similar_songs = np.flip(np.argsort(cos_sim[song_index].numpy()))[1:10]

In [23]:
df.loc[song_index]

artist                                                       ABBA
song                                      Lay All Your Love On Me
cleaned_text    i was n't jealous before we met now every woma...
genres                                                   Rock,Pop
Name: 50, dtype: object

In [24]:
odf = df.loc[similar_songs]
odf['similarity'] = cos_sim[song_index].numpy()[similar_songs]
odf

,artist,song,cleaned_text,genres,similarity
7688,Helloween,Lay All Your Love On Me,( originally performed by abba ) verse 1 : i w...,"Hip Hop,Pop",0.903757
5943,Faith Hill,Since I Laid Eyes On You,why did i show my hand now you know your wish ...,"Rock,Pop",0.806928
17498,Reo Speedwagon,Can't Lie To My Heart,i think to much about you baby you 've on my m...,"Rock,Pop",0.803351
23251,Alabama,This Loves On Me,"sittin ' alone at a table for two , when she w...","Rock,Pop",0.799155
6694,George Michael,I Can't Make You Love Me,"turn down the lights , turn down the bed turn ...","Rock,Pop",0.799121
4583,Dusty Springfield,A Love Like Yours,i broke your heart and i made you blue instead...,"Pop,Rock",0.797554
10455,Kenny Rogers,I Can't Make You Love Me,"turn down the lights , turn down the bed turn ...","Rock,Pop",0.796172
24002,Amy Grant,Lay Down,"it 's a cold , cold world that we 're walking ...","Rock,Pop",0.795585
2272,Cat Stevens,Last Love Song,if you do n't love me please do n't treat me t...,"Rock,Pop",0.794542
